In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from datetime import datetime
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import access_token
access_token.log_notebook_usage()

class SolventAverageCalculator:
    def __init__(self, df):
        self.df = df
        self.selected_solvents = {}  # Dictionary to store {solvent_no: {'data': row, 'percentage': float}}
        
        # Create widgets
        self.search_box = widgets.Text(
            value='',
            placeholder='Search by name, CAS number, or keywords',
            description='Search:',
            layout=widgets.Layout(width='50%')
        )
        
        self.search_results = widgets.Select(
            options=[],
            description='Results:',
            disabled=False,
            layout=widgets.Layout(width='70%', height='150px')
        )
        
        self.add_button = widgets.Button(
            description='Add Solvent',
            disabled=False,
            button_style='success',
            tooltip='Add selected solvent to the mixture'
        )
        
        # Create container for selected solvents and percentages
        self.selected_container = widgets.VBox(
            children=[widgets.HTML('<h4>Selected Solvents and Percentages</h4>')],
            layout=widgets.Layout(width='100%', max_height='300px', overflow='auto')
        )
        
        self.calculate_button = widgets.Button(
            description='Calculate Average',
            disabled=False,
            button_style='primary',
            tooltip='Calculate weighted average of all properties'
        )
        
        self.clear_button = widgets.Button(
            description='Clear All',
            disabled=False,
            button_style='warning',
            tooltip='Clear all selected solvents'
        )
        
        self.save_button = widgets.Button(
            description='Save Results',
            disabled=False,
            button_style='info',
            tooltip='Save results to CSV'
        )
        
        self.output_area = widgets.Output(
            layout=widgets.Layout(width='100%', height='400px', overflow='auto')
        )
        
        # Status label for percentage sum
        self.status_label = widgets.HTML(
            value='<p style="color: gray;">No solvents selected</p>'
        )
        
        # Set up event handlers
        self.search_box.observe(self.on_search_change, names='value')
        self.add_button.on_click(self.on_add_click)
        self.calculate_button.on_click(self.on_calculate_click)
        self.clear_button.on_click(self.on_clear_click)
        self.save_button.on_click(self.on_save_click)
        
        # Layout sections
        self.search_section = widgets.VBox([
            widgets.HTML('<h3>Search and Select Solvents</h3>'),
            self.search_box,
            widgets.HTML('<small><b>Search tips:</b> Try compound names (acetone), CAS numbers (67-64-1), or keywords (alcohol, acid)</small>'),
            self.search_results,
            self.add_button
        ], layout=widgets.Layout(width='500px'))
        
        self.selected_section = widgets.VBox([
            widgets.HTML('<h3>Selected Solvents</h3>'),
            self.selected_container,
            self.status_label,
            widgets.HBox([self.clear_button, self.calculate_button, self.save_button])
        ], layout=widgets.Layout(width='600px'))
        
        self.results_section = widgets.VBox([
            widgets.HTML('<h3>Results</h3>'),
            self.output_area
        ])
        
        # Main layout
        self.top_row = widgets.HBox([self.search_section, self.selected_section])
        self.main_layout = widgets.VBox([self.top_row, self.results_section])
        
    def on_search_change(self, change):
        """Enhanced search function"""
        search_term = change['new'].lower()
        if search_term:
            matches = []
            
            for _, row in self.df.iterrows():
                # Check Name column
                name = str(row.get('Name', '')).lower()
                # Check CAS column
                cas = str(row.get('CAS', '')).lower()
                # Check synonyms column
                synonyms = str(row.get('synonyms', '')).lower()
                
                # Check if any search term matches
                if (search_term in name or 
                    search_term in cas or 
                    search_term in synonyms):
                    matches.append(row)
            
            # Convert to DataFrame for easier handling
            matches_df = pd.DataFrame(matches) if matches else pd.DataFrame()
            
            # Update the search results dropdown
            options = []
            for _, row in matches_df.iterrows():
                name = row['Name']
                cas = row.get('CAS', '')
                option_text = f"{row['No.']} - {name}"
                if cas and str(cas) != 'nan':
                    option_text += f" (CAS: {cas})"
                options.append(option_text)
            
            self.search_results.options = options
        else:
            self.search_results.options = []
    
    def on_add_click(self, b):
        """Add selected solvent to the mixture"""
        if self.search_results.value:
            # Extract the solvent number from the selection
            solvent_no = int(self.search_results.value.split(' - ')[0])
            
            # Check if solvent is already in the selected list
            if solvent_no not in self.selected_solvents:
                # Get the solvent data
                solvent_data = self.df[self.df['No.'] == solvent_no].iloc[0]
                
                # Add to selected solvents with default percentage
                self.selected_solvents[solvent_no] = {
                    'data': solvent_data,
                    'percentage': 0.0
                }
                
                # Update the display
                self.update_selected_display()
    
    def create_solvent_row(self, solvent_no, solvent_info):
        """Create a row widget for a selected solvent"""
        name = solvent_info['data']['Name']
        
        # Create percentage input
        percentage_input = widgets.FloatText(
            value=solvent_info['percentage'],
            description='%:',
            style={'description_width': '30px'},
            layout=widgets.Layout(width='100px')
        )
        
        # Create remove button
        remove_button = widgets.Button(
            description='✖',
            button_style='danger',
            layout=widgets.Layout(width='40px')
        )
        
        # Create solvent label
        solvent_label = widgets.HTML(
            value=f"<b>{name}</b> (No. {solvent_no})",
            layout=widgets.Layout(width='300px')
        )
        
        # Set up event handlers
        def on_percentage_change(change):
            self.selected_solvents[solvent_no]['percentage'] = change['new']
            self.update_status()
        
        def on_remove_click(b):
            del self.selected_solvents[solvent_no]
            self.update_selected_display()
        
        percentage_input.observe(on_percentage_change, names='value')
        remove_button.on_click(on_remove_click)
        
        # Create row layout
        row = widgets.HBox([
            solvent_label,
            percentage_input,
            remove_button
        ], layout=widgets.Layout(margin='5px 0px'))
        
        return row
    
    def update_selected_display(self):
        """Update the display of selected solvents"""
        # Clear existing widgets except header
        header = widgets.HTML('<h4>Selected Solvents and Percentages</h4>')
        
        if not self.selected_solvents:
            self.selected_container.children = [header]
        else:
            # Create rows for each selected solvent
            rows = [header]
            for solvent_no, solvent_info in self.selected_solvents.items():
                row = self.create_solvent_row(solvent_no, solvent_info)
                rows.append(row)
            
            self.selected_container.children = rows
        
        self.update_status()
    
    def update_status(self):
        """Update the status label showing percentage sum"""
        if not self.selected_solvents:
            self.status_label.value = '<p style="color: gray;">No solvents selected</p>'
            return
        
        total_percentage = sum(info['percentage'] for info in self.selected_solvents.values())
        
        if abs(total_percentage - 100.0) < 0.001:  # Allow small floating point errors
            self.status_label.value = f'<p style="color: green;"><b>✓ Total: {total_percentage:.1f}% (Ready to calculate)</b></p>'
        elif total_percentage == 0:
            self.status_label.value = f'<p style="color: orange;">Total: {total_percentage:.1f}% (Enter percentages)</p>'
        else:
            self.status_label.value = f'<p style="color: red;"><b>⚠ Total: {total_percentage:.1f}% (Must equal 100%)</b></p>'
    
    def on_clear_click(self, b):
        """Clear all selected solvents"""
        self.selected_solvents = {}
        self.update_selected_display()
        with self.output_area:
            clear_output()
    
    def on_calculate_click(self, b):
        """Calculate weighted averages"""
        with self.output_area:
            clear_output()
            
            if not self.selected_solvents:
                print("❌ Please select at least one solvent.")
                return
            
            # Check if percentages sum to 100
            total_percentage = sum(info['percentage'] for info in self.selected_solvents.values())
            
            if abs(total_percentage - 100.0) > 0.001:
                print(f"❌ Error: Percentages must sum to 100%. Current total: {total_percentage:.2f}%")
                return
            
            print("✅ Calculating weighted averages...")
            print(f"Number of solvents: {len(self.selected_solvents)}")
            print(f"Total percentage: {total_percentage:.1f}%\n")
            
            # Get numeric columns for averaging
            numeric_columns = []
            for col in self.df.columns:
                if col not in ['No.', 'Name', 'CAS', 'SMILES', 'alias', 'synonyms', 'Note']:
                    # Check if column has numeric values
                    if pd.api.types.is_numeric_dtype(self.df[col]):
                        numeric_columns.append(col)
            
            # Calculate weighted averages
            results = {}
            valid_calculations = {}
            
            for col in numeric_columns:
                weighted_sum = 0
                total_weight = 0
                valid_values = 0
                
                for solvent_no, info in self.selected_solvents.items():
                    value = info['data'][col]
                    percentage = info['percentage']
                    
                    if pd.notna(value) and value != '':
                        try:
                            numeric_value = float(value)
                            weighted_sum += numeric_value * (percentage / 100.0)
                            total_weight += (percentage / 100.0)
                            valid_values += 1
                        except (ValueError, TypeError):
                            continue
                
                if valid_values > 0 and total_weight > 0:
                    results[col] = weighted_sum
                    valid_calculations[col] = f"{valid_values}/{len(self.selected_solvents)}"
                else:
                    results[col] = None
                    valid_calculations[col] = f"0/{len(self.selected_solvents)}"
            
            # Display results
            print("=" * 60)
            print("WEIGHTED AVERAGE RESULTS")
            print("=" * 60)
            
            # Show input mixture
            print("\nInput Mixture:")
            for solvent_no, info in self.selected_solvents.items():
                name = info['data']['Name']
                percentage = info['percentage']
                print(f"  {name}: {percentage:.1f}%")
            
            # Show calculated averages
            print(f"\nCalculated Weighted Averages:")
            print("-" * 40)
            
            results_data = []
            for col in sorted(results.keys()):
                if results[col] is not None:
                    print(f"{col:20s}: {results[col]:10.3f} (valid: {valid_calculations[col]})")
                    results_data.append({
                        'Property': col,
                        'Weighted_Average': results[col],
                        'Valid_Solvents': valid_calculations[col]
                    })
                else:
                    print(f"{col:20s}: {'N/A':>10s} (valid: {valid_calculations[col]})")
            
            # Store results for saving
            self.results_data = results_data
            self.mixture_data = [
                {
                    'Solvent': info['data']['Name'],
                    'Solvent_No': solvent_no,
                    'Percentage': info['percentage'],
                    'CAS': info['data'].get('CAS', 'N/A')
                }
                for solvent_no, info in self.selected_solvents.items()
            ]
            
            print(f"\n✅ Calculation complete! {len([r for r in results_data if r])} properties calculated.")
            print("\nNote: 'Valid' shows how many solvents had usable data for each property.")
    
    def on_save_click(self, b):
        """Save results to CSV"""
        if not hasattr(self, 'results_data'):
            with self.output_area:
                print("❌ Please calculate results first.")
            return
        
        # Create timestamp for filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"solvent_weighted_averages_{timestamp}.csv"
        
        try:
            with open(filename, 'w') as f:
                f.write("# Solvent Weighted Average Calculator Results\n")
                f.write(f"# Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
                
                # Save mixture composition
                f.write("# Mixture Composition\n")
                mixture_df = pd.DataFrame(self.mixture_data)
                mixture_df.to_csv(f, index=False)
                
                # Save calculated averages
                f.write("\n# Weighted Averages\n")
                results_df = pd.DataFrame(self.results_data)
                results_df.to_csv(f, index=False)
                
                # Save individual solvent data
                f.write("\n# Individual Solvent Data\n")
                solvent_details = []
                for solvent_no, info in self.selected_solvents.items():
                    row_data = info['data'].to_dict()
                    row_data['Selected_Percentage'] = info['percentage']
                    solvent_details.append(row_data)
                
                details_df = pd.DataFrame(solvent_details)
                details_df.to_csv(f, index=False)
            
            with self.output_area:
                print(f"\n💾 Results saved to: {filename}")
                
        except Exception as e:
            with self.output_area:
                print(f"\n❌ Error saving file: {e}")
    
    def display(self):
        """Display the calculator interface"""
        display(self.main_layout)

# Initialize and display the calculator
print("Loading solvent database...")
try:
    df = pd.read_csv('db.csv')
    print(f"✅ Loaded {len(df)} solvents from database")
    print("Creating Weighted Average Calculator...")
    calculator = SolventAverageCalculator(df)
    calculator.display()
    
    print("\n" + "="*50)
    print("SOLVENT WEIGHTED AVERAGE CALCULATOR")
    print("="*50)
    print("Instructions:")
    print("1. Search for solvents using the search box")
    print("2. Add solvents to your mixture")
    print("3. Enter percentages for each solvent (must sum to 100%)")
    print("4. Click 'Calculate Average' to get weighted averages")
    print("5. Save results to CSV if needed")
    print("="*50)
    
except Exception as e:
    print(f"❌ Error loading database: {e}")
    print("Make sure 'db.csv' is in the same directory as this notebook.")

Loading solvent database...
✅ Loaded 257 solvents from database
Creating Weighted Average Calculator...



SOLVENT WEIGHTED AVERAGE CALCULATOR
Instructions:
1. Search for solvents using the search box
2. Add solvents to your mixture
3. Enter percentages for each solvent (must sum to 100%)
4. Click 'Calculate Average' to get weighted averages
5. Save results to CSV if needed
